In [26]:
import pandas as pd
from pandas_datareader import data as pdr

In [27]:
def build_fred_macro(start="2013-01-01", end=None, shift_months=1):
    end = end or pd.Timestamp.today().strftime("%Y-%m-%d")

    raw = pdr.DataReader(["FEDFUNDS","DGS10","CPIAUCSL","GDP"], "fred", start, end)
    raw = raw.sort_index()

    # Convert to monthly month-end by taking the last available observation in each month
    m = raw.resample("ME").last()

    # GDP is quarterly; forward-fill on the monthly index so pct_change works
    m["GDP"] = m["GDP"].ffill()  # quarterly -> monthly
    
    m["1mo_inf_rate"] = m["CPIAUCSL"].pct_change(1, fill_method=None)
    m["1yr_inf_rate"] = m["CPIAUCSL"].pct_change(12, fill_method=None)
    m["1mo_GDP"]      = m["GDP"].pct_change(1, fill_method=None)
    m["1yr_GDP"]      = m["GDP"].pct_change(12, fill_method=None)

    macro = m[["FEDFUNDS","DGS10","1mo_inf_rate","1yr_inf_rate","1mo_GDP","1yr_GDP"]].copy()

    # Optional anti-lookahead: lag macro by 1 month
    if shift_months:
        macro = macro.shift(shift_months)

    macro = macro.reset_index().rename(columns={"DATE":"public_date", "index":"public_date"})
    return macro

In [28]:
macro = build_fred_macro()

In [29]:
macro[0:20]

,public_date,FEDFUNDS,DGS10,1mo_inf_rate,1yr_inf_rate,1mo_GDP,1yr_GDP
0,2013-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-02-28,0.14,2.02,NaN,NaN,NaN,NaN
2,2013-03-31,0.15,1.89,0.005430,NaN,0.000000,NaN
3,2013-04-30,0.14,1.87,-0.002812,NaN,0.000000,NaN
4,2013-05-31,0.15,1.70,-0.002088,NaN,0.004835,NaN
5,2013-06-30,0.11,2.16,0.000414,NaN,0.000000,NaN
6,2013-07-31,0.09,2.52,0.002380,NaN,0.000000,NaN
7,2013-08-31,0.09,2.60,0.001957,NaN,0.013459,NaN
8,2013-09-30,0.08,2.78,0.002387,NaN,0.000000,NaN
9,2013-10-31,0.08,2.64,0.000377,NaN,0.000000,NaN


In [30]:
import numpy as np
macro['1mo_GDP'] = macro['1mo_GDP'].replace(0, np.nan).ffill()

In [31]:
macro['1mo_GDP'][0:10]

0         NaN
1         NaN
2         NaN
3         NaN
4    0.004835
5    0.004835
6    0.004835
7    0.013459
8    0.013459
9    0.013459
Name: 1mo_GDP, dtype: float64

In [32]:
#macro.to_csv("data/fred_macro_cache.csv",index=False)

In [33]:
def _to_month_end_index(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.index = pd.to_datetime(out.index)
    out.index = out.index.to_period("M").asfreq("M")
    out = out.sort_index()
    out = out[~out.index.duplicated(keep="last")]
    return out

In [34]:
start="2013-01-01"
end="2025-12-31"
dp = pdr.DataReader("GDP", "fred", start, end)

In [35]:
dp

,GDP
DATE,
2013-01-01,16648.189
2013-04-01,16728.687
2013-07-01,16953.838
2013-10-01,17192.019
2014-01-01,17197.738
2014-04-01,17518.508
2014-07-01,17804.228
2014-10-01,17912.079
2015-01-01,18063.529


In [37]:
gdp = _to_month_end_index(dp).rename(columns={"GDP": "GDP"}).ffill()

In [38]:
gdp

,GDP
DATE,
2013-01,16648.189
2013-04,16728.687
2013-07,16953.838
2013-10,17192.019
2014-01,17197.738
2014-04,17518.508
2014-07,17804.228
2014-10,17912.079
2015-01,18063.529


In [39]:
gdp_ser = gdp["GDP"]

In [40]:
gdp_ser

DATE
2013-01    16648.189
2013-04    16728.687
2013-07    16953.838
2013-10    17192.019
2014-01    17197.738
2014-04    17518.508
2014-07    17804.228
2014-10    17912.079
2015-01    18063.529
2015-04    18279.784
2015-07    18401.626
2015-10    18435.137
2016-01    18525.933
2016-04    18711.702
2016-07    18892.639
2016-10    19089.379
2017-01    19280.084
2017-04    19438.643
2017-07    19692.595
2017-10    20037.088
2018-01    20328.553
2018-04    20580.912
2018-07    20798.730
2018-10    20917.867
2019-01    21111.600
2019-04    21397.938
2019-07    21717.171
2019-10    21933.217
2020-01    21751.238
2020-04    19958.291
2020-07    21704.437
2020-10    22087.160
2021-01    22680.693
2021-04    23425.910
2021-07    23982.379
2021-10    24813.600
2022-01    25250.347
2022-04    25861.292
2022-07    26336.304
2022-10    26770.514
2023-01    27216.445
2023-04    27530.055
2023-07    28074.846
2023-10    28424.722
2024-01    28708.161
2024-04    29147.044
2024-07    29511.664
2024-10 